In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("./annotated_phrases.csv")

In [3]:
df['phrase'] = df.phrase.astype(str)


In [4]:
dataset = {}

i = 0
while i < len(df):
    task_name = df.loc[i, "task_name"]
    desc_id = df.loc[i, "description_id"]

    inp = ""
    while i < len(df) and df.loc[i, "phrase_kind"] == "input":
        inp += df.loc[i, "phrase"] + " "
        i+=1

    grid_size = ""
    while i < len(df) and df.loc[i, "phrase_kind"] == "grid_size":
        grid_size += df.loc[i, "phrase"] + " "
        i+=1
    
    output = ""
    while i < len(df) and df.loc[i, "phrase_kind"] == "output":
        output += df.loc[i, "phrase"] + " "
        i+=1

    if bool(inp) == False or bool(grid_size) == False or bool(output) == False: print(task_name)
    dataset[task_name] = {"input": inp, "grid_size": grid_size, "output": output}

    while i < len(df) and df.loc[i, "task_name"] == task_name: i+=1


In [5]:
dataset

{'00d62c1b.json': {'input': 'a design with lines and empty squares. ',
  'grid_size': 'stays the same. ',
  'output': 'look at the color that form the design in the input. Find any empty areas of blocks that are enclosed  by the lines that form the main design, fill them yellow. You should end with the same original design from the input, but with the empty areas filled yellow. '},
 '017c7c7b.json': {'input': 'a grid that is 3x6 with black and blue. ',
  'grid_size': 'add 3 rows to the bottom. ',
  'output': 'keep the original pattern, change the blue to red, and then continue the pattern in the new 3 rows. '},
 '0520fde7.json': {'input': 'a grid with a gray line down the center ',
  'grid_size': 'should be 3x3 ',
  'output': 'look at both the left and right parts of the input grid. You will notice that the left and right parts are 3x3. For each square that is colored on both the left and right parts, color the output grid with red on the new 3x3. '},
 '05269061.json': {'input': '7 by 

In [6]:
import os
import sys
sys.path.append("../")
import solver

tasks_desc_not_avail = []

with open("../subset.txt", "r") as f:
    files = f.read().splitlines()

for f_name in dir(solver):
    if f_name[:5] == 'solve' and f_name[6:] not in files:
        if f_name[6:]+".json" not in dataset.keys(): tasks_desc_not_avail.append(f_name[6:])


In [7]:
len(tasks_desc_not_avail), tasks_desc_not_avail

(45,
 ['007bbfb7',
  '025d127b',
  '045e512c',
  '09629e4f',
  '1190e5a7',
  '1f0c79e5',
  '2281f1f4',
  '3345333e',
  '3428a4f5',
  '3ac3eb23',
  '444801d8',
  '447fd412',
  '469497ad',
  '46f33fce',
  '484b58aa',
  '57aa92db',
  '6aa20dc0',
  '6e19193c',
  '73251a56',
  '746b3537',
  '7837ac64',
  '8403a5d5',
  '8f2ea7aa',
  '91413438',
  '9aec4887',
  '9d9215db',
  'a78176bb',
  'ac0a08a4',
  'b527c5c6',
  'b548a754',
  'caa06a1f',
  'cbded52d',
  'cdecee7f',
  'd07ae81c',
  'd13f3404',
  'd22278a0',
  'd406998b',
  'd6ad076f',
  'd8c310e9',
  'e8dc4411',
  'eb281b96',
  'f1cefba8',
  'fafffa47',
  'fcc82909',
  'ff28f65a'])

In [8]:
# def matrix_to_string(matrix):
#     string = ""
#     for row in matrix:
#         string += " ".join([str(elem) for elem in row]) + "\n"
#     return string


# def get_demonstration_examples_prompt(task_name):
#     with open(f"../ARC-AGI/data/training/{task_name}.json", "r") as f:
#         task = json.load(f)

#     train_inputs = []
#     train_outputs = []

#     for ex in task["train"]:
#         train_inputs.append(tuple(map(tuple, ex["input"])))
#         train_outputs.append(tuple(map(tuple, ex["output"])))

#     prompt = ""
#     for i, o in zip(train_inputs, train_outputs):
#         prompt += "Input:\n"
#         prompt += matrix_to_string(i)
#         prompt += "Output:\n"
#         prompt += matrix_to_string(o)
#         prompt += "\n"
    
#     return prompt

In [ ]:
prompt = ""

for idx, (task_name, desc) in enumerate(dataset.items()):
    prompt += f"Example {idx}:\n"
    prompt += f"Required Description of program {idx}\n"
    prompt += "Input: " + desc["input"] + "\n"
    prompt += "Grid Size: " + desc["grid_size"] + "\n"
    prompt += "Output: " + desc["output"]

    prompt += "\n\n"

icl = prompt
print(len(icl))

In [8]:
import inspect

prompt = ""

for idx, (task_name, desc) in enumerate(dataset.items()):
    prompt += f"Example {idx}:\n"
    prompt += inspect.getsource(getattr(solver, "solve_"+task_name[:-5]))
    prompt += f"Required Description of program {idx}\n"
    prompt += "Input: " + desc["input"] + "\n"
    prompt += "Grid Size: " + desc["grid_size"] + "\n"
    prompt += "Output: " + desc["output"]

    prompt += "\n\n"

icl = prompt
print(len(icl))

247302


In [9]:
len(dataset)

354

In [10]:
import google.generativeai as genai

keys = [] # LIST OF KEYS
i=0


/Users/kartik/anaconda3/envs/arc_py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def generate_description(task_name):
    prompt = icl
    prompt += f"Example:\n"
    prompt += inspect.getsource(getattr(solver, "solve_"+task_name))

    prompt += f"Required Description of input-output transformation example\n"
    prompt += "Input:\n"
    prompt += "Grid Size:\n"
    prompt += "Output:\n"
    prompt += "Generate only Input, Grid Size, Output description for last example"

    global i
    genai.configure(api_key=keys[i])
    i=(i+1)%len(keys)
    generation_config = {
        "temperature": 0,
        "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
    )
    dsl_file = genai.upload_file(path="../dsl.py",
        display_name="Domain-specific Language")
    
    response = model.generate_content([dsl_file, prompt]).text
    i = response.find("Input: ") + 7
    j = response.find("Grid Size: ") + 11
    k = response.find("Output: ") + 8

    inp = response[i:j-11]    
    grid_size = response[j:k-8]    
    output = response[k:]   

    return inp, grid_size, output


In [12]:
for task in tasks_desc_not_avail:
    inp, grid_size, output = generate_description(task)
    dataset[task+".json"] = {
        "input": inp,
        "grid_size": grid_size,
        "output": output,
    }

In [14]:
program_db_desc = pd.DataFrame(columns=["task_name", "input", "grid_size", "output"], dtype=str)
for task_name, d in dataset.items():
    program_db_desc.loc[len(program_db_desc)] = [task_name, d["input"], d["grid_size"], d["output"]]

program_db_desc.to_csv("./program_db_desc.csv")

In [16]:
len(dataset)

399